 # Building some simple model

- We will build a simple model whuch uses the similarity measures.


- It is ___ might be worse than___ any of the basic prediction algorithms. (__like, THE BASELINE MODEL__, with biases)


- It is just to show that, How we can use __similarity matrices__ while recommending OR predicting the user's rating/interest in some perticular movie/item.

__ 1.  Get the training_sparse matrix from the disk__

In [58]:
# let's read movie_movie similarity matrix from the disk if if it is present
import os
from scipy import sparse

if os.path.isfile('train_sparse_matrix.npz'):
    # just read from the disk 
    print('reading it from disk..')
    train_sparse_matrix = sparse.load_npz('train_sparse_matrix.npz')
    print('Our similarity matrix is of',train_sparse_matrix.shape,' size')
else:
    # We dont't want to create one. So,
    print("Oops..!, you don't have the m_m_similarity file in your directory. Get it first..")

reading it from disk..
Our similarity matrix is of (2649430, 17771)  size


__ 2.  Get the SImilar_movies dict  from the disk__

In [116]:
import pickle

In [117]:
# let's read movie_movie similarity matrix from the disk if if it is present
import os
from scipy import sparse

if os.path.isfile('similar_movies.pickle'):
    # just read from the disk 
    print('reading it from disk..')
    with open('similar_movies.pickle', 'rb') as sm:
        similar_movies = pickle.load(sm)
    print("Done")
else:
    # We dont't want to create one. So,
    print("Oops..!, you don't have the file in your directory. Get it first..")

reading it from disk..
Done


In [118]:
similar_movies[59]

array([  867, 11602, 13112,  9894, 13224,  7530, 12893,  2664, 14693,
       14049,  4957, 14581,  2877,  9342,  7604,  5756,  1745,  2031,
       12006,  1981,  8188, 12445, 16520, 12120,  5095,  5722, 10565,
        6811,  7575,   966,  8749,  4703,  4193, 10194,  8479, 10525,
       17392, 12575,  5866,  4760, 13942,  2025,  3774,  3038, 16326,
        2258,  8332, 10207,  7345,   142,  3209,   141,  8319, 11729,
         383,  8909,   744,  2084,  8362,  1538,   685, 11018,  3899,
        5804,  2744,   979,  5491,  2501, 16595,  4322,  6039, 13026,
       13172,  4430,  9255, 14535, 13432,  5483,  8927, 13844, 10518,
       17237, 13913, 11950, 11807,  4937, 13308,  7484,  6694,  9444,
       14416,  2374,   970,   126, 13338,   602,  3618,  1680, 10705,
       16268], dtype=int64)

__ 3. Get the averages file from the disk__ 

In [70]:
if os.path.isfile('averages.pickle'):
    # open -------- read ------- close
    avgs_file = open('averages.pickle', 'rb')
    averages = pickle.load(avgs_file)
    avgs_file.close()
else:
    print("Averages.pickle file is not presnt. Get it...")

In [71]:
averages.keys()

dict_keys(['global', 'user', 'movie'])

In [72]:
# testing averages file
averages['global']

3.5516949697678895

#### 4. Get the CrossValidtion dataset from the disk

In [1]:
import pandas as pd
import numpy as np

In [26]:
if os.path.isfile('cv.csv'):
    # read from the disk
    print('reading from the disk..')
    cv_df = pd.read_csv('cv.csv')
    print('Done')
else:
    print(" No cross validation file exists. Please get it..")

reading from the disk..
Done


In [27]:
cv_df.head()

,movie,user,rating,date
0,4149,1706831,3.0,2005-03-29
1,6166,1821144,3.0,2005-03-29
2,3917,1928592,3.0,2005-03-29
3,1798,2256821,3.0,2005-03-29
4,12911,94782,3.0,2005-03-29


In [29]:
sum(cv_df.duplicated(subset=['movie','user','rating']))

0

## 1.  GARSI Algorithm ---- Using MOVIE_MOVIE similarity

( __G__lobal __A__verage __R__ating with __S__imilar __I__tems )

__Algorithm__:

 $\Large \hat r_{ui} = \frac{\text { Global Average Rating } + \frac {\sum (\text{User Ratings of similar movies})} {\text {No of ratings }}} {2} $

- We will take __top 10__ ratings of similar movies by this user, and take average of them.


- We will take this ( avg rating of similar movies by this user) and Global average of ratings, We again average them.


- It is not the best model, but, it is not that bad.

___why top 10..? why not top 20..??___

- It is just for this model. Later we will build __a Regression out of this top_10 or top_20 features__ and let the model figure it out which ratings to consider.


- We will also add some other features to this regression problem. like _user average rating_, _movie average rating_, _global average rating_, ...., ..etc.,. 

In [24]:
# It is the algorithm........

def average_of_top_similar_movies(similarities, ratings, top=100, verbose=False):
    # get all similarities into an numpy array.
    sims = similarities.toarray().ravel() if sparse.issparse(similarities) else similarities
    # get the top similar users or items, we don't care about those values...
    top_movies = np.argsort(sims)[::-1][1:top+1] # we are ignoring the movie/user itself..
    
    top_ratings = ratings.toarray().ravel() if sparse.issparse(ratings) else ratings
    if verbose:
        print(sum(top_ratings[top_users_or_movies] != 0))
        print(sum(top_ratings[top_users_or_movies]))
        print(len(top_users_or_movies))
    # return the average rating of top ''top'' users or movies
    return sum(top_ratings[top_users_or_movies]) / (sum(top_ratings[top_users_or_movies]!=0))

### Reading the data from the cv_df file and predict the rating

In [53]:
users = cv_df.user.values
movies = cv_df.movie.values

In [ ]:
for u,m in zip(users, movies):
    print(u,m)

In [36]:
users.shape[0]

20095809

In [43]:
from datetime import datetime

In [119]:
train_sparse_matrix[477189, similar_movies[886]].toarray()

array([[0., 4., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 2., 0.,
        0., 2., 3., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 4.,
        0., 0., 0., 0., 0., 4., 0., 1., 0., 0., 0., 0., 0., 5., 4., 5.,
        0., 4., 0., 5., 1., 0., 2., 0., 0., 0., 0., 0., 0., 2., 0., 2.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4.,
        5., 0., 0., 0., 4., 0., 0., 5., 0., 0., 1., 0., 0., 0., 3., 2.,
        0., 0., 0., 1.]])

In [203]:
len(similar_movies.keys())

16464

In [129]:
averages.keys()

dict_keys(['global', 'user', 'movie'])

In [220]:
start = datetime.now()
def predict_ratings(users, movies, averages, verbose=False):
    cold_start_count = 0
    predicted = list()
    count= 0
    print('predicting ratings for {} user_movie pairs...'.format(len(users)))
    for user, movie in zip(users, movies):
        # predict the rating for this (user, movie) pair
        # ratings given by this user to similar movies

        # print(user, movie)
        try:
            top_ratings = train_sparse_matrix[user, similar_movies[movie]].toarray().ravel()
            top_ratings = top_ratings[top_ratings != 0][:10]
            
            if(len(top_ratings) != 0):
                pred_rating = np.mean([np.mean(top_ratings), averages['global']])
            else :       
                # user didn't rate any of  the top 100 similar movies..
                # so just take this movie average rating
                pred_rating =  averages['movie'][movie]
                
        except KeyError:
            # It is a new User or new Movie or there are no ratings for given user for top similar movies...
            ########## Cold STart Problem ##########
            cold_start_count = cold_start_count + 1
            pred_averge = averages['global']
            
        except:
            # we just want KeyErrors to be resolved. Not every Exception...
            raise
        
        # add it to the prdicted ratings list..
        predicted.append(pred_rating)
        
        count = count + 1
        if verbose:
            if count % 200000 == 0:
                print("predicted for {} users. Time taken : {}".format(count, datetime.now() - start))
            
    print('Done')
    print(datetime.now() - start)
    print("Cold Start cases handled : ", cold_start_count)
    return predicted

predicted_ratings = predict_ratings(users = cv_df.user.values,
                                   movies=cv_df.movie.values,
                                   averages=averages, verbose=True)

predicting ratings for 20095809 user_movie pairs...
predicted for 200000 users. Time taken : 0:01:17.023165
predicted for 400000 users. Time taken : 0:02:34.083629
predicted for 600000 users. Time taken : 0:03:50.582331
predicted for 800000 users. Time taken : 0:05:06.772229
predicted for 1000000 users. Time taken : 0:06:23.748211
predicted for 1200000 users. Time taken : 0:07:41.256943
predicted for 1400000 users. Time taken : 0:08:56.326979
predicted for 1600000 users. Time taken : 0:10:13.000987
predicted for 1800000 users. Time taken : 0:11:29.491564
predicted for 2000000 users. Time taken : 0:12:43.608944
predicted for 2200000 users. Time taken : 0:13:58.797734
predicted for 2400000 users. Time taken : 0:15:14.931452
predicted for 2600000 users. Time taken : 0:16:31.591888
predicted for 2800000 users. Time taken : 0:17:47.832156
predicted for 3000000 users. Time taken : 0:19:04.315638
predicted for 3200000 users. Time taken : 0:20:20.368578
predicted for 3400000 users. Time taken 

In [162]:
train_sparse_matrix[:, 10781]

<2649430x1 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>